# Airbnb Boston Analysis

In [ ]:
# Colocar um texto falando do Airbnb e, depois, apresentar os problemas como o CRISP-DM começando com o business understanding

In [2]:
# Data understanding
# Vai ser o que está abaixo 

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline